<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/news_classifications_pytorch_kobert_ipynb_210802_9590.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
# 변경: 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 43 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595731 sha256=da5d8b15a82d744479e920bbc255c4124143011326e9008016b5b007c21a2c5f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 3.6 MB/s 
     |████████████████████████████████| 754 kB 4.0 MB/s 
     |████████████████████████████████| 895 kB 8.4 MB/s 
     |████████████████████████████████| 3.0 MB 16.5 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0zrfwzgm
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-0zrfwzgm
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=3576eda44c1336af64129aa2c3863af0faa6ca7c94a36b4f86f6faeb63b1a729
  Stored in directory: /tmp/pip-ephem-wheel-cache-fyznj7qy/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/rsn_nlp_project/sample_0728.csv')

#라벨 데이터 category 정수로 변환

encoder = LabelEncoder()
encoder.fit(dataset['category'])
dataset['category'] = encoder.transform(dataset['category'])

# possible_labels = dataset['category'].unique()
# label_dict = {}
# for index, possible_label in enumerate(possible_labels):
#     label_dict[possible_label] = index

# dataset['category'] = dataset['category'].replace(label_dict)

#이어서 contents 전처리(정제)
dataset['contents'] = dataset['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
dataset['contents'] = dataset['contents'].str.replace("[ ]{2,}"," ")

dataset.head()

,id,contents,category
0,NIRW1900000029.1,부산 기관장들 국가사업 차질 없이 이뤄내겠다 부산 는 서병수 부산시장을 비롯한 부...,4
1,NIRW1900000029.2,새해 해돋이 명소 등 트래픽 급증 이통 사 비상태세 이상 새해를 맞이해 통화량이 ...,0
2,NIRW1900000029.3,조기 대선의 해 향배 가를 대 변수 는 년 정유년 대통령 선거의 해가 시작됐다 이...,8
3,NIRW1900000029.4,달라지는 소비자제도 어린이운송차 속도제한장치 의무화 올해부터는 어린이 운송용 차량...,3
4,NIRW1900000029.5,광주 간 전화 건 대선주자들의 새해 첫날 지난 한해 정치권 비리로 너무 속상한 시...,8


In [ ]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'IT/과학',
 1: '경제',
 2: '문화',
 3: '미용/건강',
 4: '사회',
 5: '생활',
 6: '스포츠',
 7: '연예',
 8: '정치'}

In [ ]:
# test data / train data 나누기
# 데이터 수 줄이기
dataset = dataset[:30000]

train, test = train_test_split(dataset, test_size=0.3, random_state=42)
print("훈련용 data 개수:", len(train))
print("테스트용 data 개수:", len(test))
train.to_csv('/content/drive/MyDrive/rsn_nlp_project/sample_train.txt', sep = '\t' , index = False)
test.to_csv('/content/drive/MyDrive/rsn_nlp_project/sample_test.txt', sep = '\t' , index = False)

훈련용 data 개수: 21000
테스트용 data 개수: 9000


재시도

In [ ]:
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/rsn_nlp_project/sample_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/rsn_nlp_project/sample_test.txt", field_indices=[1,2], num_discard_samples=1)

In [ ]:
dataset_train[:5]

[[' 동부화재 교육환경 개선 등 베트남 현지 봉사활동 실시 동부화재와 베트남 자회사 가 교육환경 개선 안전교육 등 현지 봉사활동을 실시했다 동부화재는 지난 일부터 박 일간 동부화재 및 동부화재 베트남 현지 자회사인 임직원 및 동부 장학생 봉사단 동하리 등 여명이 베트남 북부 라이차우 지역에 위치한 개 중학교를 찾아 봉사활동을 펼쳤다고 일 밝혔다 봉사자들은 먼저 교육환경 개선을 위해 가 에 증축한 개 교실에 벽화 그리기를 진행했다 을 방문해 학생들에게 자전거 대와 헬멧을 증정하고 한국어린이 안전재단과 함께하는 자전거 안전교육 및 심폐소생술 교육을 진행했다 또 봉사자들은 한국과 베트남의 전통 민속놀이를 함께 즐길 수 있는 미니운동회를 열어 현지 중학생들과 지역주민 여명이 함께할 수 있는 축제의 장을 마련했다 이번 봉사활동에는 베트남 장학생 명이 함께 참여해 국내 장학생들이 이들의 도움으로 현지 학생들과 수월하게 소통할 수 있었다 동부화재 김강욱 베트남 법인장은 길지 않은 시간이었지만 아이들과 함께한 시간이 정말 행복했고 안전하고 즐겁게 자전거를 탔으면 한다 며 이번 봉사프로그램에 함께해주신 동부화재 임직원 및 동부문화재단 국내외 장학생 모두에게 감사의 말씀을 드린다 고 소감을 전했다 동부화재는 매년 베트남 내 장학사업 및 교육환경개선사업을 동부문화재단과 함께 수행해왔으며 올해 처음 재단 장학생들과 함께하는 현지 봉사프로그램을 진행했다 한편 동부화재는 지난 년 월 베트남 손해보험사 지분 를 인수해 베트남 현지 보험시장에 진출했다 이후 자동차보험과 방카슈랑스에 집중하여 베트남에서 시장점유율 위를 달성하고 있다 베트남에서 사업 기반을 구축하고 인도차이나반도의 사업 확장 플랫폼을 확보중이다 ',
  '1'],
 [' 카이스트 모든 색 낼 수 있는 무지개 미세입자 제조기술 개발 카이스트 는 생명화학공학과 김신현 교수와 나노과학기술대학원 신중훈 교수 충남대학교 신소재공학과 정종율 교수 공동 연구팀이 모든 색을 낼 수 있는 무지개 미세입자 기술을 개발했다고 일 밝혔다 반사색

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 128 # max_length 512, but Considering Computing Resource => set 128
batch_size = 16 # GPU CUDA problem
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 1749, 7941, 7191, 1109, 7947,  843, 1815, 2335, 5066, 2419,
        3043, 1749, 7941, 7191, 6983, 2335, 3929,  517, 5330, 1109, 7947,
         843, 3139, 5491, 1815, 5066, 2419, 7088, 3043, 7869, 1749, 7941,
        7191, 5760, 4304,  517, 7132, 2199,  517, 7127, 1749, 7941, 7191,
        2184, 1749, 7941, 7191, 2335, 5066, 3929, 7119, 3835, 2184, 1749,
        3954, 7825, 2418, 6493, 5788, 1741, 7782, 6122, 1815,  517, 6922,
        2335,  517, 6414,  517, 6011, 7389, 7005, 4329, 6896, 3564,  835,
        4257, 7823, 6116, 4446, 2419, 7088, 4838, 5439, 3803, 2261, 2418,
        6493, 7153, 2012, 1109, 7947,  843, 7088, 3567,  517, 5330,  517,
        6896, 4285, 7463, 7828,  835, 1103, 6738, 6896, 2338, 7941, 1209,
        5573, 4370, 7869,  517, 7088, 2269, 4953, 6897, 3922, 1633, 6983,
         517, 7897, 6196, 7088, 4285, 7227,    3], dtype=int32),
 array(128, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, # bert의 hidden layer 크기?
                #  num_classes=2,
                 num_classes=9, # 분류 class 크기
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes) #nn.Linear
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()  # softmax용 Loss Function <- binary classification도 해당 loss function 사용 가능

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
#얼마나 타겟의 값을 잘 맞추었는지 평가하는 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


IndexError: ignored

In [ ]:
# 테스트 문장 예측

test_sentence = '''올여름 유일한 재난 버스터 '싱크홀'이 지금껏 접하지 못한 짜릿한 볼거리로 관객을 초대한다.

2일 오후 서울 용산구 CGV아이파크몰에서 영화 '싱크홀' 언론 시사회가 열렸다. 시사 직후 온라인으로 생중계된 기자 간담회에는 배우 차승원, 김성균, 이광수, 김혜준, 권소현, 남다름과 김지훈 감독이 참석했다.

'싱크홀'은 11년 만에 마련한 내 집이 지하 500m 초대형 싱크홀로 추락하며 벌어지는 영화다. '타워'로 한국형 재난 영화의 새 지평을 연 김지훈 감독이 연출을 맡았으며 '명량', '더 테러 라이브'의 서경훈 시각특수효과(VFX) 감독이 힘을 합쳐 완성도를 높였다.

제작진은 리얼한 재난 상황을 연출하기 위해 지상 세트의 사전 제작 단계부터 심혈을 기울인 것은 물론, 싱크홀 발생 이후의 모습을 보여주고자 지하 500m 지반의 모습을 담은 대규모 암벽 세트를 만들었다. 그리고 건물이 무너지며 발생하는 흔들림을 그대로 전달하고자 짐벌 세트 위에 빌라 세트를 짓는 대규모 프로덕션을 진행했다.'''
test_label = '7'
temp_set = []
temp_set.append(test_sentence)
temp_set.append(test_label)
test_set = []
test_set.append(temp_set)

In [ ]:
test_set

[["올여름 유일한 재난 버스터 '싱크홀'이 지금껏 접하지 못한 짜릿한 볼거리로 관객을 초대한다.\n\n2일 오후 서울 용산구 CGV아이파크몰에서 영화 '싱크홀' 언론 시사회가 열렸다. 시사 직후 온라인으로 생중계된 기자 간담회에는 배우 차승원, 김성균, 이광수, 김혜준, 권소현, 남다름과 김지훈 감독이 참석했다.\n\n'싱크홀'은 11년 만에 마련한 내 집이 지하 500m 초대형 싱크홀로 추락하며 벌어지는 영화다. '타워'로 한국형 재난 영화의 새 지평을 연 김지훈 감독이 연출을 맡았으며 '명량', '더 테러 라이브'의 서경훈 시각특수효과(VFX) 감독이 힘을 합쳐 완성도를 높였다.\n\n제작진은 리얼한 재난 상황을 연출하기 위해 지상 세트의 사전 제작 단계부터 심혈을 기울인 것은 물론, 싱크홀 발생 이후의 모습을 보여주고자 지하 500m 지반의 모습을 담은 대규모 암벽 세트를 만들었다. 그리고 건물이 무너지며 발생하는 흔들림을 그대로 전달하고자 짐벌 세트 위에 빌라 세트를 짓는 대규모 프로덕션을 진행했다.",
  '7']]

In [ ]:
test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=9)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)

  model.eval() # 평가 모드로 변경
      
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
  print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 9 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


tensor([[-0.7541, -0.7598,  5.4711, -0.3886, -1.2535, -0.5007, -0.6052,  0.1452,
         -0.5176]], device='cuda:0', grad_fn=<AddmmBackward>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



epoch 4 test acc 1.7708703374777974



In [ ]:
torch.save(model, '/content/drive/MyDrive/rsn_nlp_project/20210802model_95.pt')